In [1]:
import numpy
import matplotlib.pyplot as plt

In [2]:
def cosine(x):
    return (numpy.cos(3 * x) + 1) / 2

def gaussian(x):
    return numpy.exp(-0.25 * x ** 2)

def polynomial(x):
    return (x ** 5 + 3 * x ** 4 - 11 * x ** 3 - 27 * x ** 2 + 10 * x + 64) / 100 

def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))
    
def y(x, w):
    return w.T.dot(x)

def loss_function(y, t): # L2 Loss
    return numpy.mean((y - t) ** 2) / 2

def compute_gradient_layer_1(x, w, h, y, t):
    tmp = (y - t)
    print(type(tmp))
    
    hadamard = w * h * (1 - h)
    
    print(hadamard)
    print(hadamard.shape)
    
    print(tmp * hadamard)
    tmp_2 = np.outer(tmp, x)
    return 2 * numpy.mean(tmp_2) 

def compute_gradient_layer_2(h, y, t):
    return 2 * numpy.mean((y - t) * h)

def gradient_descent(x, t, w_1, w_2, eta, epochs):
    loss_array = numpy.zeros(epochs)
    
    for epoch in range(epochs):
        h = sigmoid(w_1.dot(x))
        Y = y(h, w_2)
        w_1 -= eta * compute_gradient_layer_1(x, w_2, h, Y, t)
        w_2 -= eta * compute_gradient_layer_2(h, Y, t)
        h = sigmoid(w_1.dot(x))
        Y = y(h, w_2)
        loss_array[epoch] = loss_function(Y, t)
    
    return w_1, w_2, loss_array

In [3]:
# define number of samples
d = numpy.random.randint(low=20, high=2000)

# datasets
x_1 = numpy.random.uniform(low=-2, high=2, size=(d+1))
x_1[0] = 1. # add bias
t_1 = cosine(x_1)

x_2 = numpy.random.uniform(low=-2, high=2, size=(d+1))
x_2[0] = 1. # add bias
t_2 = gaussian(x_2)

x_3 = numpy.random.uniform(low=-4.5, high=3.5, size=(d+1))
x_3[0] = 1. # add bias
t_3 = polynomial(x_3)

# init weights
k = 10 # number of hidden neurons
w_1 = numpy.random.uniform(low=-10, high=10, size=(k+1,d+1)) # first layer weights
w_1[0,:] = 0 # set to zero (bias)
w_2 = numpy.random.uniform(low=-10, high=10, size=(k+1)) # second layer weights

# learning rate
eta = 0.02

# number of epochs
epochs = 1

# run gradient descent for x_1 dataset
w_x_1_1, w_x_1_2, loss_1 = gradient_descent(x_1, t_1, numpy.copy(w_1), numpy.copy(w_2), eta, epochs)

# plot the graph

# run gradient descent for x_3 dataset
#w_x_2_1, w_x_2_2, loss_2 = gradient_descent(x_2, t_2, numpy.copy(w_1), numpy.copy(w_2), eta, epochs)

# plot the graph

# run gradient descent for x_3 dataset
#w_x_3_1, w_x_3_2, loss_3 = gradient_descent(x_3, t_3, numpy.copy(w_1), numpy.copy(w_2), eta, epochs)

# plot the graph




<class 'numpy.ndarray'>
[ 1.27443080e+000 -6.93369360e-207  5.80999360e-076 -2.28544986e-029
  1.32509810e-018  0.00000000e+000 -1.36909801e-055  0.00000000e+000
 -1.74525412e-007  0.00000000e+000  0.00000000e+000]
(11,)


ValueError: operands could not be broadcast together with shapes (715,) (11,) 